In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
from sklearn.model_selection import train_test_split

# Load the Data

In [11]:
data = pd.read_csv('train.En.csv')
valid = pd.read_csv('task_A_En_test.csv')

In [26]:
data = data[['tweet', 'sarcastic']]
data.rename(columns={'tweet': 'text'}, inplace=True)
data

,text,sarcastic
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1
...,...,...
3463,The population spike in Chicago in 9 months is...,0
3464,You'd think in the second to last English clas...,0
3465,I’m finally surfacing after a holiday to Scotl...,0
3466,Couldn't be prouder today. Well done to every ...,0


In [31]:
train, test = train_test_split(data)

In [32]:
train

,text,sarcastic
2717,"#IdLikeToGetPaidFor working full time, 12.5 hr...",0
626,Just almost got hit on the highway by a @Cityo...,1
3000,Motivation Wednesday Night:\r\n\r\nI AM the po...,0
561,They really said you can’t have nice weather A...,1
2512,"Excited to present some research, listen to so...",0
...,...,...
2474,my worst enemy is the lil machine at the eye d...,0
1841,If they are going after the misspelling “defin...,0
591,I'm just living off salads and yogurt and lovi...,1
2231,Really want to see the goggleboxers watching B...,0


In [33]:
test

,text,sarcastic
1395,Some pretty great things are happening at the ...,0
1103,i just deleted every tweet from beyond this ye...,0
380,"@kagune_kun despicable tweet, my liege",1
2947,locals saying “not all cops” are missing the p...,0
1078,The most wonderful time of the year ✨ in my el...,0
...,...,...
2490,pcc is cheugy,0
1271,Just ordered the 'Sea-Lover's Sharing menu' fo...,0
738,Why was I not told Matthew Stafford and Cooper...,1
2892,I love how Javer only started to suspect Jean ...,0


In [34]:
valid

,text,sarcastic
0,"Size on the the Toulouse team, That pack is mo...",0
1,Pinball!,0
2,So the Scottish Government want people to get ...,1
3,villainous pro tip : change the device name on...,0
4,I would date any of these men 🥺,0
...,...,...
1395,I’ve just seen this and felt it deserved a Ret...,0
1396,Omg how an earth is that a pen !!! 🤡,0
1397,Bringing Kanye and drake to a tl near you,0
1398,"I love it when women are referred to as ""girl ...",1


# Evaluate

In [36]:
from sklearn.metrics import f1_score

## Testing Randomness

In [41]:
f1_score(np.random.randint(0, 2, test['sarcastic'].shape), test['sarcastic'])

0.3647604327666152